In [1]:
import torch
import torchaudio
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [2]:
print("command..")
device = torch.device("cpu")

command = """
    You are an intelligent scheduling assistant. Your job is to convert short, informal instructions into polite and professional messages that can be sent directly to the recipient to request a service or schedule an appointment. 
    
    Analyze the input to extract key details:
     - Time and date of the meeting or service.
     - Recipient type (e.g., barber, doctor, restaurant, etc.).
     - Service requested (if provided).

    Construct a polite and concise message based on this information. The tone should be courteous and professional.

    Examples:
    Input: 3PM today barber
    Output: Hi, I would like to schedule a haircut appointment today at 3 PM. Is this time available?

    Guidelines:
    If the input does not specify all details, make a general inquiry.

    For example:
    Input: Barber appointment
    Output: Hello, I’d like to book a haircut appointment. What times are available?

    Always keep the tone polite and professional.
    
    So now please take an appointment with the barber for me at 3 PM today. generate just one sentence.\n"""

tokenizer = AutoTokenizer.from_pretrained("facebook/MobileLLM-1B", use_fast=False)
gen_config = GenerationConfig(do_sample=False, top_p=1, temperature=1)

model = AutoModelForCausalLM.from_pretrained("facebook/MobileLLM-1B", trust_remote_code=True).to(device)
input_ids = tokenizer(command, return_tensors="pt").input_ids
output = model.generate(input_ids, max_length=500)

# tts
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
# take only the response of the model
# output_text = output_text.split("Output: ")[3]
print("\n\n\n")
# take the last sentence from the response
print("\n\nTHE GENERATED TEXT IS: ")
print(output_text)
print("\n\n\n")
output_text = output_text.split("Output: ")[-1]


print("\n\n\n")
print("\n\nTHE GENERATED SPLITTED TEXT IS: ")
print(output_text)

del output
del model
del tokenizer

from whisperspeech.pipeline import Pipeline

pipe = Pipeline(s2a_ref='collabora/whisperspeech:s2a-q4-tiny-en+pl.model')

audio = pipe.generate(output_text)

del pipe

torchaudio.save("output.wav", audio.cpu(), 16000)

import sounddevice as sd

# run the audio



print("\n\n\n")
print("recording the barber response...")
fs = 16000
duration = 5  # seconds
myrecording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
sd.wait()

import soundfile as sf
sf.write('response.wav', myrecording, fs)
# s2t
waveform, sample_rate = torchaudio.load("response.wav")
myrecording = waveform[0].numpy()

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")


inputs = processor(myrecording, sampling_rate=16000, return_tensors="pt")

generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

del model
del processor
del inputs
del generated_ids
print("\n\n\n")
print("transcription.. " + transcription[0])
# llama
prompt= "please extract the response yes or No for this audio trasncription\n" + transcription[0]

tokenizer = AutoTokenizer.from_pretrained("facebook/MobileLLM-1B", use_fast=False)
gen_config = GenerationConfig(do_sample=False, top_p=1, temperature=1)

model = AutoModelForCausalLM.from_pretrained("facebook/MobileLLM-1B", trust_remote_code=True).to(device)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

output = model.generate(input_ids, max_length=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

del tokenizer
del model
del input_ids
del output

command..


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
Some weights of the model checkpoint at facebook/MobileLLM-1B were not used when initializing MobileLLMForCausalLM: ['lm_head.weight']
- This IS expected if you are initializing MobileLLMForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileLLMForCausalLM from the







THE GENERATED TEXT IS: 
 
    You are an intelligent scheduling assistant. Your job is to convert short, informal instructions into polite and professional messages that can be sent directly to the recipient to request a service or schedule an appointment. 
    
    Analyze the input to extract key details:
     - Time and date of the meeting or service.
     - Recipient type (e.g., barber, doctor, restaurant, etc.).
     - Service requested (if provided).

    Construct a polite and concise message based on this information. The tone should be courteous and professional.

    Examples:
    Input: 3PM today barber
    Output: Hi, I would like to schedule a haircut appointment today at 3 PM. Is this time available?

    Guidelines:
    If the input does not specify all details, make a general inquiry.

    For example:
    Input: Barber appointment
    Output: Hello, I’d like to book a haircut appointment. What times are available?

    Always keep the tone polite and professional

/home/iliass/.local/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/iliass/.local/lib/python3.11/site-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(






recording the barber response...


Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2024-11-23 14:07:27.218856: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-23 14:07:27.490381: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-23 14:07





transcription.. i know that just then i may give them the moment


Some weights of the model checkpoint at facebook/MobileLLM-1B were not used when initializing MobileLLMForCausalLM: ['lm_head.weight']
- This IS expected if you are initializing MobileLLMForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileLLMForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


please extract the response yes or No for this audio trasncription
i know that just then i may give them the moment of the audio
i will give them the moment of the audio
i will give them the moment of the audio and the response yes or no
i will give them the moment of the audio and the response yes or no for this audio trasncription
i will give them the moment of the audio and the response yes or no for this audio trasncription and
